In [1]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep as para
from time import time as contar
import datetime as dt

import pandas as pd
import os
import re

from openpyxl import load_workbook, Workbook

In [2]:
class Navegacao():
    def __init__(self):
        options=webdriver.ChromeOptions()
        if input('Deseja esconder o navegador? y/n\n')=='y': 
            options.add_argument("--headless")
            self.navegador = webdriver.Chrome(options=options)
        else: self.navegador = webdriver.Chrome()
        self.vars = {'contas': [],
                    'link do afim': '',
                    'ignored_exceptions': (NoSuchElementException,StaleElementReferenceException), 
                    'senha': f'Jaspekde27.{dt.date.today().month*3}'}
        self.pl = {}

    def espera_login(self, ano):
        if ano > 2018:
            self.navegador.find_element(By.ID, 'user').send_keys('034.410.422-20')
            self.navegador.find_element(By.ID, 'password').send_keys(self.vars['senha'])
            self.navegador.find_element(By.ID, 'password').send_keys(Keys.ENTER)
        else:
            self.navegador.find_element(By.NAME, 'user').send_keys('03441042220')
            self.navegador.find_element(By.NAME, 'password').send_keys(self.vars['senha'])
            self.navegador.find_element(By.NAME, 'password').send_keys(Keys.ENTER)

    def esperar_elementos_carregarem(self, tipo_elemento: str, nome_elemento: str, varios_elementos: bool = False, tempo_maximo_espera: int = 30):
        """Espera um ou vários elementos carregarem na página

        Args:
            tipo_elemento (str): tipo do elemento
            nome_elemento (str): nome do elemento
            varios_elementos (bool): procurar mais de um elemento
            tempo_maximo_espera (int): tempo determinado para executar a espera, em segundos. Padrão 30 (0,5 min).
        """    
        tempo_inicial = contar() 
        
        if varios_elementos:
            elementos = WebDriverWait(self.navegador, tempo_maximo_espera, ignored_exceptions=self.vars['ignored_exceptions'])\
                            .until(expected_conditions.presence_of_all_elements_located((tipo_elemento, nome_elemento)))
            tempo_final = contar()
            # print(f'>>> Elementos "{nome_elemento}" encontrados em {round(tempo_final - tempo_inicial, 2)}s! (len({len(elementos)}))')
            return elementos
        else:
            elemento = WebDriverWait(self.navegador, tempo_maximo_espera, ignored_exceptions=self.vars['ignored_exceptions'])\
                            .until(expected_conditions.presence_of_element_located((tipo_elemento, nome_elemento)))
            tempo_final = contar()
            if len(elemento.text) > 20: elemento_texto_resumido = elemento.text[:20]
            else: elemento_texto_resumido = elemento.text
                
            # print(f'>>> Elemento "{elemento_texto_resumido}" encontrado em {round(tempo_final - tempo_inicial, 2)}s!')
            return elemento

    def entra_AFIM(self, afim):
        """Abre o AFIM

        Args:
            afim (_type_): Ano no qual o FIM irá abrir
        """
        print(f'\n=============== Executando entra_AFIM {afim}! ===============')
        
        if afim <= 2018:
            link = f'http://afim2.manaus.am.gov.br/AfimPRD{afim}/'
            self.navegador.get(link)
            self.espera_login(afim)
            para(1)
            self.navegador.switch_to.window(self.navegador.window_handles[-1])

        #AFIM DE 2019
        if afim == 2019:
            link = 'https://afim1.manaus.am.gov.br/AfimPRD2019/'
            self.navegador.get(link)
            self.espera_login(afim)
            para(2)
            for w in self.navegador.window_handles:
                self.navegador.switch_to.window(w)
                if 'mensagem' in self.navegador.current_url: self.navegador.close()
                elif str(afim) in self.navegador.current_url: pass
                else: self.navegador.close()
                
            self.navegador.switch_to.window(self.navegador.window_handles[-1])

        if afim >= 2020:
            link = f'https://afim1.manaus.am.gov.br/AfimPRD{afim}/'
            self.navegador.get(link)
            
            self.espera_login(afim)
            self.navegador.switch_to.window(self.navegador.window_handles[-1])
            para(1)

        self.vars['link do afim'] = link

        print(f'=============== Finalizando entra_AFIM {afim}! ===============\n')
        para(2)

    def NL_detalhada(self, obj_NL):
        """Pesquisa a Nota de Lançamento desejada para retirar as informações

        Args:
            obj_NL (_type_): Objeto Selenium da Nota de Lançamento

        Returns:
            _type_: Tupla com as informações extraidas
        """
        if not 'NL' in obj_NL.text: 
            print(f'\n=============== Impossível executar! Não é uma NL ({obj_NL.text}) ===============')
            return ('', '', '', '', '', '','')
        
        
        print(f'\n=============== Executando a NL ({obj_NL.text}) ===============')
        para(2)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        elemento = obj_NL
        ActionChains(self.navegador)\
        .double_click(elemento)\
        .perform()

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        credor_empresa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'tr td input[name="txtCredor"]').get_attribute('value')
        obs = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'textarea[name="txtObservacao"]').text
        NE = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtNuNe"').get_attribute('value')
        natureza_despesa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNaturezaDespesa"]').get_attribute('value')
        doc = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNuDocmento"').get_attribute('value')
        processo = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNuProcesso"').get_attribute('value')
        
        self.navegador.close()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        
        return (obj_NL.text.replace(' ', ''), credor_empresa, obs, NE, natureza_despesa, doc, processo)
    
    def NL_detalhada_2020(self, obj_NL):
        """Pesquisa a Nota de Lançamento desejada para retirar as informações

        Args:
        obj_NL (_type_): Objeto Selenium da Nota de Lançamento

        Returns:
        _type_: Tupla com as informações extraidas
        """
        if not 'NL' in obj_NL.text: 
            print(f'\n=============== Impossível executar! Não é uma NL ({obj_NL.text}) ===============')
            return ('', '', '', '', '', '','')

        print(f'\n=============== Executando a NL ({obj_NL.text}) ===============')
        para(2)
        obj_NL.click()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        credor_empresa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, '[name="txtCredor"]', ).get_attribute('value')
        obs = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'textarea[name="txtObservacao"]').text
        NE = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtNuNe"').get_attribute('value')
        natureza_despesa = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNaturezaDespesa"]').get_attribute('value')
        doc = self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[name="txtRdNuDocmento"').get_attribute('value')
        processo = self.esperar_elementos_carregarem(By.CSS_SELECTOR, '[name="txtRdNuProcesso"]',tempo_maximo_espera=3).text


        self.navegador.close()
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        # print(obj_NL.text.replace(' ', ''), credor_empresa, obs, NE, natureza_despesa, doc, processo)
        return (obj_NL.text.replace(' ', ''), credor_empresa, obs, NE, natureza_despesa, doc, processo)

    def RAZAO(self, ano):
        print(f'\n=============== Executando RAZAO {ano}! ===============')

        # Ano anterior a 2020
        lisne_link = self.vars['link do afim'] + 'Razao.do'
        self.navegador.get(lisne_link)

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])
        self.vars['contas'] = [cbo.get_attribute('Value') for cbo in self.navegador.find_elements(By.CSS_SELECTOR, "select[name='cboConta'] option[value^='1232105']")]
        para(1)
    
    def contas_unidas(self, ano):
        for pl in self.pl:
            self.pl[pl].insert(0, "Conta Contábil", [pl for i in range(len(self.pl[pl]))],True)
            display(self.pl[pl][:1])
            
        result = pd.concat([self.pl[pl] for pl in self.pl])
        path = f"123210500\\123210500-BENS DE USO COMUM DO POVO - {ano}.xlsx"

        print(f'Salvando as contas em uma planilha excel em "{path}"')
        result.to_excel(path, index=False)

    def balancete(self):
        print(f'=============== Inicializando balancete! ===============\n')
        # if os.path.exists(r):
        #     print("O arquivo já existe.")
        #     return 0

        balancete_link = self.vars['link do afim'] + 'Balancete.do'
        self.navegador.get(balancete_link)

        para(1)
        self.navegador.switch_to.window(self.navegador.window_handles[-1])

        self.vars['contas'] = [l.get_attribute('value') for l in self.navegador.find_elements(By.CSS_SELECTOR, '[value^="1232105"]')]

        # clicando no radio "Unidade Gestora/Gestão"
        self.esperar_elementos_carregarem(By.CSS_SELECTOR, 'input[value="2"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys('270101 - Secretaria Municipal de Infraestrutura')
        self.navegador.find_element(By.CSS_SELECTOR, 'option[value="00001"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboMes"] option[value="12"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboValores"] option[value="A"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, f'select[name="cboContaIni"] option[value="{self.vars["contas"][0]}"]').click()
        self.navegador.find_element(By.CSS_SELECTOR, f'select[name="cboContaFim"] option[value="{self.vars["contas"][-1]}"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboTipo"] option[value="2"]').click()
        if not self.navegador.find_element(By.CSS_SELECTOR, 'input[name="ssoOpcao"]').is_selected(): self.navegador.find_element(By.CSS_SELECTOR, 'input[name="ssoOpcao"]').click()

        self.navegador.find_element(By.CSS_SELECTOR, '[id="IMGprocurar"]').click()
        para(3)
        self.extraindo_balancete()
        print(f'=============== Finalizando balancete! ===============\n')

    def extraindo_balancete(self):
        print('========== Executando extraindo_balancete! ==========')
        balancete = {}
        for i in [e.text for e in self.navegador.find_elements(By.CSS_SELECTOR, 'tr > th.label') if e.text!='']: balancete[i] = []

        keys = list(balancete.keys())

        linhas = [l for l in self.navegador.find_elements(By.CSS_SELECTOR, 'tr tr[align]')]
        for linha in linhas: 
            l = [l for l in linha.text.split('  ') if l!='']
            # print('l:',l)
            if '1232105' in l[0]: 
                conta = l[0]
                # print('var conta:',conta)
            elif not '-' in l[0]:
                balancete[keys[0]].append(f"{conta} ({l[0].replace(' ', '')})")
                for index in range(1,6): balancete[keys[index]].append(int(l[index].replace('.', '').replace(',','')))
                balancete[keys[6]].append(l[6])
                # print('=== contas ===')
                # display(balancete)
        # print(len(contas))
        self.vars['balancete'] = pd.DataFrame(balancete)
        display(self.vars['balancete'])
        print('========== Finalizando extraindo_balancete! ==========')

    def razao_lista(self, ano):
        print(f'========== Inicializando razao_lista! ({ano}) ==========')
        
        # Selecionando as datas
        print(f'Selecionando a Data Inicial: 01/01/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataInicio').get_attribute('value')!=f'01/01/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'img0').click()
            while not (str(ano) in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
            if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Janeiro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Janeiro')
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[0].click()
        else: print('Data Inicial já selecionada')

        print(f'Selecionando a Data Final: 31/12/{ano}')
        if self.esperar_elementos_carregarem(By.ID, 'mkdDataFim').get_attribute('value')!=f'31/12/{ano}':
            self.esperar_elementos_carregarem(By.ID, 'img1').click()
            while not (str(ano) in self.navegador.find_elements(By.CSS_SELECTOR, "font[size='1']")[0].text): self.navegador.find_element(By.ID, "toLeft").click()
            if self.navegador.find_element(By.CSS_SELECTOR, "#month [selected]").text!='Dezembro': self.esperar_elementos_carregarem(By.CSS_SELECTOR, "#month").send_keys('Dezembro')
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, "td[class='calReg']", True, 30)[-1].click()
        else: print('Data Final já selecionada')

        dic = {}
        for c in range(len(self.vars['balancete'])):

            nome_da_conta = self.vars['balancete'].iloc[c,0]
            conta_contabil = re.search(r'\s?(\d.*) \(', nome_da_conta).group(1)
            conta_corrente = re.search(r'\s\((.*)\)', nome_da_conta).group(1)

            print(conta_contabil, conta_corrente)

            '''Preechendo as informações da tela razão'''
            self.esperar_elementos_carregarem( By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').send_keys("270101 - Secretaria Municipal de Infraestrutura")
            self.navegador.find_element(By.CSS_SELECTOR, 'select[name="cboUnidadeGestora"]').click()

            self.esperar_elementos_carregarem(By.NAME, 'cboConta').send_keys(conta_contabil)
            self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#txtContaCorrente').send_keys(conta_corrente)

            try: self.esperar_elementos_carregarem(By.ID, 'IMGprocurar').click()
            except: self.esperar_elementos_carregarem(By.ID, 'btnProcurar').click()

            '''Captura as informações do RAZAO para criar uma tabela'''
            lista_no_razao = []
            for n in self.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr'): lista_no_razao.append(n.text.split('  '))

            lista_no_razao = list(zip(*lista_no_razao))

            """Cria o DataFrame a partir da lista de tuplas zip, especificando os nomes das colunas e eliminando as linhas desnecessarias"""
            df = pd.DataFrame(lista_no_razao).transpose()

            df = df.set_axis(df.iloc[0], axis=1, copy=False)
            df = df.drop(len(df)-1).drop(0).drop(1)
            df = df.reset_index(drop=True)

            """Cria um DataFrame a partir das informações retiradas de cada NL"""
            detalhes_da_NL = [['Número','CREDOR','OBSERVAÇÃO','NOTA DE EMPENHO',
                    'NATUREZA DE DESPESA','N° DO DOCUMENTO', 'PROCESSO']]

            for i in self.navegador.find_elements(By.CSS_SELECTOR, '#dbGrid tr td[onclick="selectCell(this,3)"]'): 
                if i.text!=' ': detalhes_da_NL.append(self.NL_detalhada(i))

            df2 = pd.DataFrame(list(zip(*detalhes_da_NL))).transpose()
            df2.columns = df2.iloc[0][:len(df2.columns)]

            df_mescle = df2.merge(df, on='Número')
            df_mescle = df_mescle.drop(['UG', 'Gestão', 'Saldo', 'D/C'], axis=1)
            self.vars[nome_da_conta] = df_mescle

            ''' Unindo os dataframes em um dicionario'''
            df_mescle['Conta Contábil'] = nome_da_conta
            dic[nome_da_conta] = df_mescle

        ''' Exportando os dataframes em excel'''
        df_unidos = pd.concat(dic, ignore_index=True)
        df_unidos.to_excel(f'123210500-Bens de Uso Comum do Povo ({ano}).xlsx', index=False)

        print(f'========== Finalizando razao_lista! ({ano}) ==========')

    def razao_lista_2020(self, ano):
        print(f'========== Inicializando razao_lista! ({ano}) ==========')
        self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#mkdDataInicio').click()
        self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#mkdDataFim').click()

        self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#mkdDataInicio').click()
        # Selecionando a data inicial
        self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="month current"]')[0].click()
        # Selecionando o ano
        para(1)
        while self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="year current"]')[0].text != str(ano): self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="year prev"]')[0].click()
        # Selecionando o mês de "Janeiro"
        self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="months"] li')[0].click()
        # Selecionando o dia 1
        if self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class=""]')[0].text=='1': self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class=""]')[0].click()
        else: self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class="picked"]')[0].click()


        # Selecionando a data final
        self.esperar_elementos_carregarem(By.CSS_SELECTOR, '#mkdDataFim').click()
        self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="month current"]')[-1].click()
        # Selecionando o ano
        while self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="year current"]')[-1].text != str(ano): self.navegador.find_elements(By.CSS_SELECTOR, '[data-view="year prev"]')[-1].click()
        # Selecionando o mês de "Dezembro"
        self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="months"] li')[-1].click()
        # Selecionando o dia 31
        if self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class=""]')[-1].text=='31': self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class=""]')[-1].click()
        else: self.navegador.find_elements(By.CSS_SELECTOR, 'ul[data-view="days"] li[class="picked"]')[-1].click()

        dic = {}
        for c in range(len(self.vars['balancete'])):

            nome_da_conta = self.vars['balancete'].iloc[c,0]
            conta_contabil = re.search(r'\s?(\d.*) \(', nome_da_conta).group(1)
            conta_corrente = re.search(r'\s\((.*)\)', nome_da_conta).group(1)

            print(conta_contabil, conta_corrente)

            self.navegador.find_element( By.CSS_SELECTOR, '#cboUnidadeGestora').send_keys("270101 - Secretaria Municipal de Infraestrutura")
            self.navegador.find_element(By.CSS_SELECTOR, '#cboConta').send_keys(conta_contabil)
            para(2)
            self.navegador.find_element(By.CSS_SELECTOR, '#txtContaCorrente').clear()
            self.navegador.find_element(By.CSS_SELECTOR, '#txtContaCorrente').send_keys(conta_corrente)

            try: self.esperar_elementos_carregarem(By.ID, 'IMGprocurar',tempo_maximo_espera=10).click()
            except: self.esperar_elementos_carregarem(By.ID, 'btnProcurar',tempo_maximo_espera=10).click()

            while self.navegador.find_element(By.CSS_SELECTOR, 'p#hourglass').text=='Carregando': pass

            rows = [r.text.split(' ') for r in self.navegador.find_elements(By.CSS_SELECTOR,'tr[role="row"]') if r'/' in r.text]
            df = pd.DataFrame(rows)
            df = df.set_axis(df.iloc[0], axis=1, copy=False)
            if len(df)<=1: pass
            else: 
                df = df.drop(len(df)-1).drop(0).drop(1)
                df = df.reset_index(drop=True)

                detalhes_da_NL = [['Número','CREDOR','OBSERVAÇÃO','NOTA DE EMPENHO',
                                    'NATUREZA DE DESPESA','N° DO DOCUMENTO', 'PROCESSO']]

                self.navegador.switch_to.window(self.navegador.window_handles[-1])
                for i in self.navegador.find_elements(By.CSS_SELECTOR, 'a.conta-corrente[onclick^="detalharPorNu"]'): detalhes_da_NL.append(self.NL_detalhada_2020(i))

                df2 = pd.DataFrame(list(zip(*detalhes_da_NL))).transpose()
                df2.columns = df2.iloc[0][:len(df2.columns)]
                df2 = df2.drop(0)
                df2 = df2.reset_index(drop=True)

                df_mescle = df2.merge(df, on='Número')
                df_mescle = df_mescle.drop(['UG', 'Gestão', 'Saldo', 'D/C'], axis=1)
                self.vars[nome_da_conta] = df_mescle

                df_mescle['Conta Contábil'] = nome_da_conta
                dic[nome_da_conta] = df_mescle

        ''' Exportando os dataframes em excel'''
        df_unidos = pd.concat(dic, ignore_index=True)
        df_unidos.to_excel(f'123210500-Bens de Uso Comum do Povo ({ano}).xlsx', index=False)

        print(f'========== Finalizando razao_lista! ({ano}) ==========')



In [3]:
ano = 2014
N = Navegacao()
N.entra_AFIM(ano)
N.balancete()
N.RAZAO(ano)
# N.razao_lista_2020(ano)
self = N


=============== Executando entra_AFIM 2022! ===============
=============== Finalizando entra_AFIM 2022! ===============

=============== Inicializando balancete! ===============

========== Executando extraindo_balancete! ==========


,Conta Contábil,Saldo Inicial,Saldo Mês Anterior,Debito Até Mês,Credito Até Mês,Saldo Atual,D/C
0,123210501-Infraestrutura Serv. Terceiros (@),64267095,64267095,0,0,64267095,D
1,123210502-Ruas (@),57117874883,59138870622,2380751462,359755723,59138870622,D
2,123210503-Infraestrutura : massa asfalta (@),7692714545,7692714545,0,0,7692714545,D
3,123210504-Sistema De Drenagem (@),33361766469,43125973948,25307958739,14538559417,44131165791,D
4,123210506-Infraestrutura : serv. de usinagem d...,39341289169,53032582868,27699503077,11728466737,55312325509,D
5,123210517-Obras de Infraestrutura de revitaliz...,90407865779,119287328886,76960131247,42275585543,125092411483,D
6,123210519-Obras de Urbanização e Consrvação do...,18972614911,19302698958,1650420205,1320336158,19302698958,D
7,123210520-Obras de Infra estrutura para Copa d...,4125483271,4125483271,0,0,4125483271,D
8,123210523-Praças (@),145730900,145730900,0,0,145730900,D
9,123210580-Estudos e Projetos P/ Infraestrutura...,821878803,821878803,0,0,821878803,D


========== Finalizando extraindo_balancete! ==========
=============== Finalizando balancete! ===============


=============== Executando RAZAO 2022! ===============


# Imprime a balancete


def tabela_print(pl, aba, salvar=False):
    print(aba.max_column, aba.max_row)
    for j in range(1, aba.max_row + 1):
        for i in range(1, aba.max_column + 1):
            print(aba.cell(row = j, column = i).value, end='     ')
        print('') 

    if salvar: pl.save('tt.xlsx')

#### Formata as planilhas:
##### - Adiciona a coluna Conta Contábil
##### - Ajusta a coluna Movimento para negativo de acordo com a coluna Evento

In [6]:
dados = pd.read_excel(f"123210500-Bens de Uso Comum do Povo ({ano}).xlsx")

# Verificar e converter a coluna para o tipo de dados string
dados['Evento'] = dados['Evento'].astype(str)

# Criar a nova coluna "N° Conta Contábil"
dados['N° Conta Contábil'] = dados['Conta Contábil'].str.split('-').str[0]

dados.loc[dados['Evento'].str[2] == '5', ['Movimento']] = '-' + dados.loc[dados['Evento'].str[2] == '5', ['Movimento']]
dados.to_excel(f"123210500-Bens de Uso Comum do Povo ({ano}).xlsx", index=False)


##### - Ajusta as colunas com seu tipos corretos

In [8]:
ano = 2014
for i in range(1):
    dados = pd.read_excel(f"123210500-Bens de Uso Comum do Povo ({ano}).xlsx")

    print(dados.dtypes)
    for coluna in list(dados .columns):
        # print(coluna)
        if coluna == 'Movimento' and not '-' in dados[coluna]:
            print(dados[coluna])
            dados[coluna] = dados[coluna].str.replace('.','')
            dados[coluna] = dados[coluna].str.replace(',','.')

            # Converter a coluna 'Valor' para o formato de moeda
            dados[coluna] = dados[coluna].astype(float)

            # Arredondar os valores da coluna 'Valor' para duas casas decimais (caso necessário)
            dados[coluna] = dados[coluna].round(2)

        else: dados[coluna] = dados[coluna].astype(str)
        # dados['PROCESSO'] = dados['PROCESSO'].astype(str)
    dados.to_excel(f"123210500-Bens de Uso Comum do Povo ({ano})-test.xlsx", index=False)



Número                  object
CREDOR                  object
OBSERVAÇÃO              object
NOTA DE EMPENHO         object
NATUREZA DE DESPESA     object
N° DO DOCUMENTO         object
PROCESSO               float64
 Data                   object
Evento                   int64
Movimento               object
Conta Contábil          object
dtype: object
0      156.842,11
1      111.752,52
2      206.687,54
3      275.830,00
4       40.600,00
          ...    
641     -74191.54
642     74.191,54
643    130.200,00
644   -1180381.54
645   -1966112.08
Name: Movimento, Length: 646, dtype: object
